In [1]:
!pip install -U tensorflow

In [2]:
import tensorflow as tf
print("TensorFlow:", tf.__version__)

TensorFlow: 2.13.0


## 텐서 와 연산

In [3]:
#스칼라 텐서를 생성
t1 = tf.constant(42)
print(t1)
#배열을 가지고 생성
t2 = tf.constant([[1, 2, 3], [4, 5, 6]])
print(t2)

tf.Tensor(42, shape=(), dtype=int32)
tf.Tensor(
[[1 2 3]
 [4 5 6]], shape=(2, 3), dtype=int32)


In [4]:
#인덱싱
print(t2[:, 1:]) #행전체 그리고 열은 1번 부터 끝까지 선택
print(t2[:, 1]) #첫번째 열만 선택 - 일차원 배열로 생성
print(t2[:, 1, tf.newaxis]) #첫번째 열만 선택 - 다차원 배열로 생성
#tf.newaxis를 추가하면 차원이 하나 추가됩니다.

tf.Tensor(
[[2 3]
 [5 6]], shape=(2, 2), dtype=int32)
tf.Tensor([2 5], shape=(2,), dtype=int32)
tf.Tensor(
[[2]
 [5]], shape=(2, 1), dtype=int32)


In [5]:
#연산
print(t2 + 10) #broadcast 연산 수행
print(tf.square(t2))
print(t2 @ tf.transpose(t2))

tf.Tensor(
[[11 12 13]
 [14 15 16]], shape=(2, 3), dtype=int32)
tf.Tensor(
[[ 1  4  9]
 [16 25 36]], shape=(2, 3), dtype=int32)
tf.Tensor(
[[14 32]
 [32 77]], shape=(2, 2), dtype=int32)


In [6]:
from tensorflow import keras
K = keras.backend
K.square(K.transpose(t2)) + 10

<tf.Tensor: shape=(3, 2), dtype=int32, numpy=
array([[11, 26],
       [14, 35],
       [19, 46]])>

In [7]:
import numpy as np
ar = np.array([2, 3, 4])
print(np.sum(ar))

#numpy 배열을 가지고 Tensor 생성
x = tf.constant(ar)
print(x)
#Tensor를 numpy 배열로 변경
print(type(x.numpy()))

#numpy 와 Tensor 연산에 데이터 공유 가능
print(tf.square(ar))
print(np.square(x))

9
tf.Tensor([2 3 4], shape=(3,), dtype=int32)
<class 'numpy.ndarray'>
tf.Tensor([ 4  9 16], shape=(3,), dtype=int32)
[ 4  9 16]


In [8]:
# 자료형이 달라서 에러 발생
try:
    tf.constant(10) + tf.constant(10.3)
except tf.errors.InvalidArgumentError as ex:
    print(ex)

cannot compute AddV2 as input #1(zero-based) was expected to be a int32 tensor but is a float tensor [Op:AddV2] name: 


In [9]:
try:
    print(tf.cast(tf.constant(10), tf.float32) + tf.constant(10.3))
except tf.errors.InvalidArgumentError as ex:
    print(ex)

tf.Tensor(20.3, shape=(), dtype=float32)


In [10]:
#변수 생성
v = tf.Variable([[1, 2, 3], [4, 5, 6]])
print(v)
v.assign(5 + v)
print(v)
v[0,0].assign(42)
print(v)

<tf.Variable 'Variable:0' shape=(2, 3) dtype=int32, numpy=
array([[1, 2, 3],
       [4, 5, 6]])>
<tf.Variable 'Variable:0' shape=(2, 3) dtype=int32, numpy=
array([[ 6,  7,  8],
       [ 9, 10, 11]])>
<tf.Variable 'Variable:0' shape=(2, 3) dtype=int32, numpy=
array([[42,  7,  8],
       [ 9, 10, 11]])>


## 문자열 Tensor

In [11]:
tf.constant("안녕하세요") #하나의 기본 자료형으로 인코딩 해서 저장


<tf.Tensor: shape=(), dtype=string, numpy=b'\xec\x95\x88\xeb\x85\x95\xed\x95\x98\xec\x84\xb8\xec\x9a\x94'>

In [12]:
#Tensor 배열 - 실수 3개를 저장하는 텐서 배열을 생성
array = tf.TensorArray(dtype=tf.float32, size=3)
#3개의 데이터 저장
array = array.write(0, tf.constant([1.0, 2.0]))
array = array.write(1, tf.constant([3.0, 4.0]))
array = array.write(2, tf.constant([5.0, 6.0]))
#첫번째 데이터 읽기
print(array.read(1))

tf.Tensor([3. 4.], shape=(2,), dtype=float32)


In [13]:
#ragged tensor: 각 배열의 길이가 달라도 됩니다.
ar = tf.ragged.constant([[10, 20], [30, 40, 50], [50, 60]])
print(ar)

<tf.RaggedTensor [[10, 20], [30, 40, 50], [50, 60]]>


In [14]:
#희소 텐서 생성
#3행 4열의 행렬을 만들고 indices 자리에 values의 값을 순서대로 배정한
#희소 행렬
s = tf.SparseTensor(indices=[[0,1], [1, 0], [2, 3]],
                             values = [1.0, 2.0, 3.0],
                    dense_shape=[3, 4])
print(tf.sparse.to_dense(s))

tf.Tensor(
[[0. 1. 0. 0.]
 [2. 0. 0. 0.]
 [0. 0. 0. 3.]], shape=(3, 4), dtype=float32)


In [15]:
s1 = tf.constant([[1, 2, 3, 1, 1], [2, 4, 5, 8, 11]])
s2 = tf.constant([[1, 2, 3, 4, 4], [2, 7, 5, 8, 9]])
print(s1)
print(s2)
print(tf.sparse.to_dense(tf.sets.union(s1, s2)))

# set 연산은 배열로 리턴하기 때문에 연산의 결과 각 배열의 데이터 개수가 
# 다르면 뒤에 0을 채워 동일한 개수로 맞춤

tf.Tensor(
[[ 1  2  3  1  1]
 [ 2  4  5  8 11]], shape=(2, 5), dtype=int32)
tf.Tensor(
[[1 2 3 4 4]
 [2 7 5 8 9]], shape=(2, 5), dtype=int32)
tf.Tensor(
[[ 1  2  3  4  0  0  0]
 [ 2  4  5  7  8  9 11]], shape=(2, 7), dtype=int32)


In [16]:
#노드 생성
node1 = tf.constant(3.0)
node2 = tf.constant(4.0)

#그래프 실행
print(node1)
print(node2)



tf.Tensor(3.0, shape=(), dtype=float32)
tf.Tensor(4.0, shape=(), dtype=float32)


In [17]:
#텐서플로우 코드 최적화가 가능하도록 작성
@tf.function
def tf_cube(x):
  print(x)#값의 변화가 없는 구문이므로 함수 호출전에 전부 수행해버림
  return x ** 3 #텐서플로우 함수 호출 구문으로 변경되서 실행

result = tf_cube(tf.constant(2.0))
print(result)

Tensor("x:0", shape=(), dtype=float32)
tf.Tensor(8.0, shape=(), dtype=float32)


In [18]:
#2행 5열의 난수 발생: 0에서 1사이에서 균등한 분포로
result = tf.random.uniform([2, 5], 0.0, 1.0)
print(result)

tf.Tensor(
[[0.42557883 0.7682241  0.25939333 0.5936333  0.31286085]
 [0.81325924 0.44810474 0.5325515  0.8918463  0.8842479 ]], shape=(2, 5), dtype=float32)


In [19]:
#2행 5열의 난수 발생: 0에서 1사이에서 정규 분포에서 평균은 0
#표준 편차는 1
result = tf.random.normal([2, 5], 0.0, 1.0)
print(result)

tf.Tensor(
[[ 0.81143385  0.22757065 -0.5357656   0.6337147  -1.4835889 ]
 [ 0.00664213  0.9732755   0.78386855 -0.02731042  0.797204  ]], shape=(2, 5), dtype=float32)


## 뉴런 생성

In [20]:
# 활성화 함수 - 시그모이드
import math

def sigmoid(x):
  return 1 / (1 + math.exp(-x))

In [21]:
#입력 값
X = 1.0
#타겟
y = 0.0
#가중치를 랜덤하게 설정
w = tf.random.normal([1], 0, 1)
#출력
output = sigmoid(X * w)
print("예측값:", output)

예측값: 0.538491953057512


In [22]:
#경사 하강법
for i in range(1000):
  output = sigmoid(X * w)
  error = y - output
  w = w + X * 0.1 * error #학습률을 0.1로 설정
  if i % 50 == 0:
    print(i, output, error)

0 0.538491953057512 -0.538491953057512
50 0.18877082681496266 -0.18877082681496266
100 0.10384024405742118 -0.10384024405742118
150 0.07030852512429557 -0.07030852512429557
200 0.05281867298968098 -0.05281867298968098
250 0.0421800533328842 -0.0421800533328842
300 0.03505725701513762 -0.03505725701513762
350 0.02996661607139489 -0.02996661607139489
400 0.026152533508177676 -0.026152533508177676
450 0.02319108527217034 -0.02319108527217034
500 0.020826644473282602 -0.020826644473282602
550 0.018896074366783103 -0.018896074366783103
600 0.017290548110247227 -0.017290548110247227
650 0.015934705460977444 -0.015934705460977444
700 0.014774728558460053 -0.014774728558460053
750 0.013771193829752415 -0.013771193829752415
800 0.012894573857269108 -0.012894573857269108
850 0.012122322297161217 -0.012122322297161217
900 0.011436891195069765 -0.011436891195069765
950 0.010824451227195577 -0.010824451227195577


In [23]:
#경사 하강법 문제 발생
#가중치 만을 곱하게 되는 경우 Input이 0이면 가중치가 변경되지 않음
X = 0
y = 1
w = tf.random.normal([1], 0, 1)
for i in range(1000):
  output = sigmoid(X * w)
  error = y - output
  w = w + X * 0.1 * error #학습률을 0.1로 설정
  if i % 50 == 0:
    print(i, output, error)

0 0.5 0.5
50 0.5 0.5
100 0.5 0.5
150 0.5 0.5
200 0.5 0.5
250 0.5 0.5
300 0.5 0.5
350 0.5 0.5
400 0.5 0.5
450 0.5 0.5
500 0.5 0.5
550 0.5 0.5
600 0.5 0.5
650 0.5 0.5
700 0.5 0.5


750 0.5 0.5
800 0.5 0.5
850 0.5 0.5
900 0.5 0.5
950 0.5 0.5


In [24]:
#경사 하강법 문제 발생 - 편향을 추가해서 해결
X = 0
y = 1
w = tf.random.normal([1], 0, 1)

b = tf.random.normal([1], 0, 1)
for i in range(1000):
  output = sigmoid(X * w + 1 * b)
  error = y - output
  w = w + X * 0.1 * error #학습률을 0.1로 설정
  b = b + 1 * 0.1 * error
  if i % 50 == 0:
    print(i, output, error)

0 0.5014378090804075 0.49856219091959253
50 0.8197539274988886 0.18024607250111135
100 0.8990420504551506 0.10095794954484938


150 0.9310706673934117 0.06892933260658829
200 0.9479771452361517 0.05202285476384827
250 0.958334305030315 0.041665694969684974
300 0.9653012805073421 0.0346987194926579
350 0.9702970536233178 0.02970294637668225
400 0.9740492668321767 0.02595073316782326
450 0.9769682033594201 0.02303179664057986
500 0.9793022018475982 0.020697798152401803
550 0.9812102470796213 0.018789752920378655
600 0.9827986412157099 0.017201358784290077
650 0.9841411741441842 0.015858825855815772
700 0.9852906036612812 0.014709396338718816
750 0.9862856611404566 0.013714338859543429
800 0.9871553115785231 0.012844688421476902
850 0.9879218187103793 0.012078181289620682
900 0.9886024457963097 0.011397554203690263
950 0.9892108227643202 0.010789177235679848


In [25]:
#AND를 뉴런으로 구현
import numpy as np

X = np.array([[0,0], [0,1], [1,0], [1,1]])
y = np.array([[0], [0], [0], [1]])
b = tf.random.normal([1], 0, 1)
#입력이 2개라서 가중치를 2개 생성
w = tf.random.normal([2], 0, 1)

for i in range(2000):
  #오차의 합계를 저장할 변수
  error_sum = 0
  #모든 입력을 대입해서 수행
  for j in range(4):
    output = sigmoid(np.sum(X[j] * w) + 1 * b)
    error = y[j][0] - output
    w = w + X[j] * 0.1 * error
    b = b + 1 * 0.1 * error
    error_sum += error
  if i % 100 == 0:
    print(i, error_sum)

0 -0.47962801392561827


100 -0.1797909222069185
200 -0.11387954323031324
300 -0.08424023716427698
400 -0.06690416710117458
500 -0.05543893935144706
600 -0.047280830152248654
700 -0.041182277805761114
800 -0.03645281600249889
900 -0.032681586151091666
1000 -0.02960479000982108
1100 -0.02704957617191059
1200 -0.02489374627447255
1300 -0.02305127706602747
1400 -0.021459306251021687
1500 -0.02007100153940515
1600 -0.0188485978065822
1700 -0.017764998429640334
1800 -0.01679620574221704
1900 -0.015929090048830863


In [26]:
#predict
for i in range(4):
  print("X:", X[i], "Y:", y[i], "Predict:", sigmoid(np.sum(X[i]*w)+b))

X: [0 0] Y: [0] Predict: 2.4311250864094812e-05
X: [0 1] Y: [0] Predict: 0.025208668743221214
X: [1 0] Y: [0] Predict: 0.02528614282519339
X: [1 1] Y: [1] Predict: 0.9650284966786367


### OR를 경사하강법으로 구현

In [27]:
#OR를 뉴런으로 구현
import numpy as np

X = np.array([[0,0], [0,1], [1,0], [1,1]])
y = np.array([[0], [1], [1], [1]])
b = tf.random.normal([1], 0, 1)
#입력이 2개라서 가중치를 2개 생성
w = tf.random.normal([2], 0, 1)

for i in range(2000):
  #오차의 합계를 저장할 변수
  error_sum = 0
  #모든 입력을 대입해서 수행
  for j in range(4):
    output = sigmoid(np.sum(X[j] * w) + 1 * b)
    error = y[j][0] - output
    w = w + X[j] * 0.1 * error
    b = b + 1 * 0.1 * error
    error_sum += error
  if i % 100 == 0:
    print(i, error_sum)

#predict
for i in range(4):
  print("X:", X[i], "Y:", y[i], "Predict:", sigmoid(np.sum(X[i]*w)+b))

0 1.944105506329858


100 -0.08267360829377202
200 -0.04685231501937534
300 -0.03263436875244918
400 -0.025014682652575893
500 -0.020249666659787904
600 -0.016989476659453523
700 -0.014620678735418516
800 -0.012823153951536728
900 -0.011414198943576849
1000 -0.010280559989673567
1100 -0.00934931377613394
1200 -0.008570541738736837
1300 -0.007910344744713714
1400 -0.007344076132442708
1500 -0.006852262890474019
1600 -0.0064215550546496
1700 -0.006041567029450738
1800 -0.005703929454827252
1900 -0.005400740085320335
X: [0 0] Y: [0] Predict: 0.025439665233400858
X: [0 1] Y: [1] Predict: 0.9898907490134178
X: [1 0] Y: [1] Predict: 0.9898413378561349
X: [1 1] Y: [1] Predict: 0.9999972640774104


In [28]:
#XOR를 뉴런으로 구현
import numpy as np

X = np.array([[0,0], [0,1], [1,0], [1,1]])
y = np.array([[0], [1], [1], [0]])
b = tf.random.normal([1], 0, 1)
#입력이 2개라서 가중치를 2개 생성
w = tf.random.normal([2], 0, 1)

for i in range(2000):
  #오차의 합계를 저장할 변수
  error_sum = 0
  #모든 입력을 대입해서 수행
  for j in range(4):
    output = sigmoid(np.sum(X[j] * w) + 1 * b)
    error = y[j][0] - output
    w = w + X[j] * 0.1 * error
    b = b + 1 * 0.1 * error
    error_sum += error
  if i % 100 == 0:
    print(i, error_sum)

#predict
for i in range(4):
  print("X:", X[i], "Y:", y[i], "Predict:", sigmoid(np.sum(X[i]*w)+b))

0 -1.77966826323197
100 -0.018776421201785243
200 -0.0037927469785519197
300 -0.0007648365475975138
400 -0.000154208891475216
500 -3.110510420867296e-05
600 -6.259952074039177e-06
700 -1.2583203661264974e-06
800 -2.699060452915347e-07
900 -7.445683847251416e-09
1000 -6.980329025907395e-08
1100 -6.980329025907395e-08
1200 -6.980329025907395e-08
1300 -6.980329025907395e-08
1400 -6.980329025907395e-08
1500 -6.980329025907395e-08
1600 -6.980329025907395e-08
1700 -6.980329025907395e-08
1800 -6.980329025907395e-08
1900 -6.980329025907395e-08
X: [0 0] Y: [0] Predict: 0.5128177040587629
X: [0 1] Y: [1] Predict: 0.5000000363215804
X: [1 0] Y: [1] Predict: 0.48718236853665897
X: [1 1] Y: [0] Predict: 0.4743815336379531


In [29]:
X = np.array([[0,0], [0,1], [1,0], [1,1]])
y = np.array([[0], [1], [1], [0]])

model = tf.keras.Sequential([
    tf.keras.layers.Dense(units=2, activation='sigmoid',
                          input_shape=(2,)),
    tf.keras.layers.Dense(units=1, activation='sigmoid')
])

model.compile(optimizer=tf.keras.optimizers.SGD(lr=0.01), loss='mse')

model.summary()

history = model.fit(X, y, epochs=1000, batch_size=1)

model.predict(X)

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 2)                 6         
                                                                 
 dense_1 (Dense)             (None, 1)                 3         
                                                                 
Total params: 9 (36.00 Byte)
Trainable params: 9 (36.00 Byte)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
Epoch 1/1000


4/4 [==============================] - 0s 0s/step - loss: 0.2558
Epoch 2/1000
4/4 [==============================] - 0s 5ms/step - loss: 0.2557
Epoch 3/1000
4/4 [==============================] - 0s 0s/step - loss: 0.2557
Epoch 4/1000
4/4 [==============================] - 0s 1ms/step - loss: 0.2557
Epoch 5/1000
4/4 [==============================] - 0s 0s/step - loss: 0.2557
Epoch 6/1000
4/4 [==============================] - 0s 0s/step - loss: 0.2556
Epoch 7/1000
4/4 [==============================] - 0s 0s/step - loss: 0.2556
Epoch 8/1000
4/4 [==============================] - 0s 0s/step - loss: 0.2556
Epoch 9/1000
4/4 [==============================] - 0s 5ms/step - loss: 0.2556
Epoch 10/1000
4/4 [==============================] - 0s 7ms/step - loss: 0.2556
Epoch 11/1000
4/4 [==============================] - 0s 5ms/step - loss: 0.2555
Epoch 12/1000
4/4 [==============================] - 0s 5ms/step - loss: 0.2555
Epoch 13/1000
4/4 [==============================] - 0s 1ms/step - l

array([[0.49222133],
       [0.44478187],
       [0.55111545],
       [0.5043483 ]], dtype=float32)

In [30]:
for weight in model.weights:
    print(weight)

# 가중치는 Layer 만큼 생성이 되는데

<tf.Variable 'dense/kernel:0' shape=(2, 2) dtype=float32, numpy=
array([[1.0725429 , 0.14671323],
       [0.06310132, 0.8880583 ]], dtype=float32)>
<tf.Variable 'dense/bias:0' shape=(2,) dtype=float32, numpy=array([-0.01831633, -0.00682924], dtype=float32)>
<tf.Variable 'dense_1/kernel:0' shape=(2, 1) dtype=float32, numpy=
array([[ 1.108219 ],
       [-0.9969151]], dtype=float32)>
<tf.Variable 'dense_1/bias:0' shape=(1,) dtype=float32, numpy=array([-0.08339669], dtype=float32)>


### Tensorflow Data API

In [31]:
#메모리에서 데이터 생성
X = tf.range(10)
dataset = tf.data.Dataset.from_tensor_slices(X)
print(dataset)

dataset = tf.data.Dataset.range(10)
print(dataset)
for item in dataset:
  print(item)

dataset = tf.data.Dataset.range(10)
dataset = dataset.repeat(3).batch(8, drop_remainder=True) #3번 반복하고 8개씩 묶어서 8개가 안되는 것은 버림
for item in dataset:
  print(item)

<_TensorSliceDataset element_spec=TensorSpec(shape=(), dtype=tf.int32, name=None)>
<_RangeDataset element_spec=TensorSpec(shape=(), dtype=tf.int64, name=None)>
tf.Tensor(0, shape=(), dtype=int64)
tf.Tensor(1, shape=(), dtype=int64)
tf.Tensor(2, shape=(), dtype=int64)
tf.Tensor(3, shape=(), dtype=int64)
tf.Tensor(4, shape=(), dtype=int64)
tf.Tensor(5, shape=(), dtype=int64)
tf.Tensor(6, shape=(), dtype=int64)
tf.Tensor(7, shape=(), dtype=int64)
tf.Tensor(8, shape=(), dtype=int64)
tf.Tensor(9, shape=(), dtype=int64)
tf.Tensor([0 1 2 3 4 5 6 7], shape=(8,), dtype=int64)
tf.Tensor([8 9 0 1 2 3 4 5], shape=(8,), dtype=int64)
tf.Tensor([6 7 8 9 0 1 2 3], shape=(8,), dtype=int64)


In [32]:
#셔플링
#0 ~ 29까지의 숫자를 가지고 텐서를 생성
dataset = tf.data.Dataset.range(30)
#버퍼의 크기는 5로 설정하고 seed는 42로 고정해서 일정하게 배치한 후 6개씩 묶어서 사용
dataset = dataset.shuffle(buffer_size=5, seed=42).batch(6)
for item in dataset:
  print(item)

tf.Tensor([0 2 3 6 7 9], shape=(6,), dtype=int64)
tf.Tensor([ 4  5 10  1 11  8], shape=(6,), dtype=int64)
tf.Tensor([16 15 14 18 17 21], shape=(6,), dtype=int64)
tf.Tensor([22 23 20 25 24 12], shape=(6,), dtype=int64)
tf.Tensor([27 28 19 29 13 26], shape=(6,), dtype=int64)


In [33]:
#데이터를 읽어서 나누어 저장한 후 읽어오기
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.datasets import fetch_california_housing

housing = fetch_california_housing()

#train-훈련, valid-훈련 중 검증, test-모델을 훈련한 후 확인
X_train_full, X_test, y_train_full, y_test = train_test_split(housing.data, 
                                                    housing.target.reshape(-1, 1),
                                                    random_state=42)
X_train, X_valid, y_train, y_valid = train_test_split(X_train_full, y_train_full,
                                                      random_state=42)

scaler = StandardScaler()
scaler.fit(X_train)

X_mean = scaler.mean_
X_std = scaler.scale_
print(X_mean)
print(X_std)

[ 3.89175860e+00  2.86245478e+01  5.45593655e+00  1.09963474e+00
  1.42428122e+03  2.95886657e+00  3.56464315e+01 -1.19584363e+02]
[1.90927329e+00 1.26409177e+01 2.55038070e+00 4.65460128e-01
 1.09576000e+03 2.36138048e+00 2.13456672e+00 2.00093304e+00]


In [34]:
#데이터를 여러 개의 파일로 나누어서 저장하는 함수
#매개변수는 데이터, 파일의 맨 앞의 공통된 이름, 헤더 포함 여부, 파일의 개수
def save_to_multiple_csv_files(data, name_prefix, header=None, n_parts=10):
  #파일을 저장할 디렉토리 생성
  housing_dir = os.path.join("Datasets", "housing")
  os.makedirs(housing_dir, exist_ok=True)
  #파일 경로 패턴 생성
  path_format = os.path.join(housing_dir, "my_{}_{:02d}.csv")

  filepaths = [] #파일 경로를 저장할 리스트
  m = len(data) #데이터의 개수

  #m 개수 만큼 숫자열을 만든 후 n_parts 개수 만큼 분할해서 
  #분할한 그룹 번호는 file_idx 에 행 번호는 row_indices에 저장
  for file_idx, row_indices in enumerate(np.array_split(np.arange(m), n_parts)):
    #실제 파일 경로 생성
    part_csv = path_format.format(name_prefix, file_idx)
    filepaths.append(part_csv)

    #데이터 기록
    with open(part_csv, "wt", encoding='utf-8') as f:
      if header is not None:
        f.write(header)
        f.write("\n")
        #행 인덱스를 가지고 데이터를 순회하면 각 데이터를 ,로 구분해서 작성
        for row_idx in row_indices:
          f.write(",".join([repr(col) for col in data[row_idx]]))
          f.write("\n")
  return filepaths

In [35]:
import os

In [36]:
train_data = np.c_[X_train, y_train]
valid_data = np.c_[X_valid, y_valid]
test_data = np.c_[X_test, y_test]

headers_cols = housing.feature_names + ["MedianHouseValue"]
header = ",".join(headers_cols)

train_filepaths = save_to_multiple_csv_files(train_data, "train", header, n_parts=20)
valid_filepaths = save_to_multiple_csv_files(valid_data, "valid", header, n_parts=10)
test_filepaths = save_to_multiple_csv_files(test_data, "test", header, n_parts=10)

#파일 이름을 랜덤하게 가져오기
filepath_Dataset = tf.data.Dataset.list_files(train_filepaths)
for filepath in filepath_Dataset:
  print(filepath)


tf.Tensor(b'Datasets\\housing\\my_train_04.csv', shape=(), dtype=string)
tf.Tensor(b'Datasets\\housing\\my_train_18.csv', shape=(), dtype=string)
tf.Tensor(b'Datasets\\housing\\my_train_16.csv', shape=(), dtype=string)
tf.Tensor(b'Datasets\\housing\\my_train_01.csv', shape=(), dtype=string)
tf.Tensor(b'Datasets\\housing\\my_train_00.csv', shape=(), dtype=string)
tf.Tensor(b'Datasets\\housing\\my_train_13.csv', shape=(), dtype=string)
tf.Tensor(b'Datasets\\housing\\my_train_05.csv', shape=(), dtype=string)
tf.Tensor(b'Datasets\\housing\\my_train_11.csv', shape=(), dtype=string)
tf.Tensor(b'Datasets\\housing\\my_train_09.csv', shape=(), dtype=string)
tf.Tensor(b'Datasets\\housing\\my_train_06.csv', shape=(), dtype=string)
tf.Tensor(b'Datasets\\housing\\my_train_17.csv', shape=(), dtype=string)
tf.Tensor(b'Datasets\\housing\\my_train_19.csv', shape=(), dtype=string)
tf.Tensor(b'Datasets\\housing\\my_train_07.csv', shape=(), dtype=string)
tf.Tensor(b'Datasets\\housing\\my_train_03.csv', sh

In [37]:
pip install tensorflow_datasets

  Obtaining dependency information for tensorflow_datasets from https://files.pythonhosted.org/packages/a1/73/7a9ed7935f6833d73b32f1e2a1210082f5ccb95445440b4e2b0f66ab7792/tensorflow_datasets-4.9.3-py3-none-any.whl.metadata
  Using cached tensorflow_datasets-4.9.3-py3-none-any.whl.metadata (9.3 kB)
  Obtaining dependency information for array-record from https://files.pythonhosted.org/packages/93/8e/c6b646029a9c544ecd6806c6ea0efb67d0ba3300e1d68518f58d9774d3f0/array_record-0.4.1-py310-none-any.whl.metadata
  Using cached array_record-0.4.1-py310-none-any.whl.metadata (503 bytes)
  Obtaining dependency information for etils[enp,epath,etree]>=0.9.0 from https://files.pythonhosted.org/packages/4a/6a/d58ec120f5e4babbf5001c144266ba623dcdae8e81dc6cdb422a98d0e0ce/etils-1.4.1-py3-none-any.whl.metadata
  Using cached etils-1.4.1-py3-none-any.whl.metadata (5.9 kB)
  Using cached promise-2.3-py3-none-any.whl
  Obtaining dependency information for tensorflow-metadata from https://files.pythonhosted.

In [38]:
import tensorflow_datasets as tfds

Datasets = tfds.load(name='mnist')
mnist_train, mnist_test = Datasets['train'], Datasets['test']

plt.figure(figsize=(6, 3))

mnist_train = mnist_train.repeat(5).batch(32).prefetch(1)
#32개를 가져오고 미리 한 묶음 더

for item in mnist_train:
  images = item['image']
  labels = item["label"]

  for index in range(5):
    plt.subplot(1, 5, index+1)
    image = images[index, ..., 0]
    label = labels[index].numpy()
    plt.imshow(image, cmap='binary')
    plt.title(label)
    plt.axis("off")
  break

ModuleNotFoundError: No module named 'resource'